# using deep learning
We will start by importing the necessary libraries

In [1]:
import tensorflow as tf
from sklearn.metrics import f1_score

2023-05-09 16:21:40.387900: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-09 16:21:40.666574: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-09 16:21:40.668193: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-09 16:21:42.683746: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import tensorflow.keras as keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [3]:
df=pd.read_csv("./test_reviews.csv")
reviews = df[:20000]
test_reviews=df[20000:]

In [4]:
neg = 0
pos = 0
neu = 0
for i in range(0,19999):
    if(reviews['Sentiment'][i]=='positive') :
        pos=pos+1
    elif (reviews['Sentiment'][i]=='negative'):
        neg=neg+1
    elif(reviews['Sentiment'][i]=='neutral') :
        neu=neu+1
pos

9946

In [5]:
reviews.head()

,Reviews,Sentiment
0,<START please give this one a miss br br <UNK>...,negative
1,<START this film requires a lot of patience be...,positive
2,<START many animation buffs consider <UNK> <UN...,positive
3,<START i generally love this type of movie how...,negative
4,<START like some other people wrote i'm a die ...,positive


In [6]:
test_reviews.head()

,Reviews,Sentiment
20000,<START the concept of this movie is pretty com...,negative
20001,<START i really liked the far cry game nice gr...,negative
20002,<START when young frances <UNK> <UNK> goes to ...,positive
20003,<START attractive husband and wife writing tea...,negative
20004,<START being a huge die hard monkey island fan...,positive


In [7]:
word_index=pd.read_csv("./word_indexes.csv")

The word index file contains mapping from words to integers.

In [8]:
word_index.head()

,Words,Indexes
0,tsukino,52009
1,nunnery,52010
2,sonja,16819
3,vani,63954
4,woods,1411


Next we are going to convert the word_index dataframe into a python dictionary so that we can use it for converting our reviews from string to integer format.

In [9]:
word_index=dict(zip(word_index.Words,word_index.Indexes))

In [10]:
word_index["<PAD>"]=0
word_index["<START"]=1
word_index["<UNK>"]=2
word_index["<UNUSED>"]=3

Now we define a function review_encoder that encodes the reviews into integer format according to the mapping specified by word_index file.

In [11]:
def review_encoder(text):
  arr=[word_index[word] for word in text]
  return arr

We split the reviews from their corresponding sentiments so that we can preprocess the reviews and sentiments separately and then later pass it to our model.

In [12]:
train_data,train_labels=reviews['Reviews'],reviews['Sentiment']
test_data, test_labels=test_reviews['Reviews'],test_reviews['Sentiment']

Before transforming the reviews as integers we need to tokenize or split the review on the basis of whitespaces
For eg.the string "The movie was wonderful" becomes ["The" , "movie" , "was" , "wonderful" ].

In [13]:
train_data=train_data.apply(lambda review:review.split())
test_data=test_data.apply(lambda review:review.split())

Since we have tokenized the reviews now we can apply the review_encoder function to each review and transform the reviews into integer format.

In [14]:
train_data=train_data.apply(review_encoder)
test_data=test_data.apply(review_encoder)

In [15]:
def encode_sentiments(x):
  if x=='positive':
    return 1
  else:
    return 0

train_labels=train_labels.apply(encode_sentiments)
test_labels=test_labels.apply(encode_sentiments)

In [16]:
train_data=keras.preprocessing.sequence.pad_sequences(train_data,value=word_index["<PAD>"],padding='post',maxlen=500)
test_data=keras.preprocessing.sequence.pad_sequences(test_data,value=word_index["<PAD>"],padding='post',maxlen=500)

In [17]:
model=keras.Sequential([keras.layers.Embedding(10000,16,input_length=500),
                        keras.layers.GlobalAveragePooling1D(),
                        keras.layers.Dense(16,activation='relu'),
                        keras.layers.Dense(1,activation='sigmoid')])

In [18]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [19]:
#training the model
history=model.fit(train_data,train_labels,epochs=50,batch_size=512,validation_data=(test_data,test_labels))

Epoch 1/50
40/40 [==============================] - 2s 38ms/step - loss: 0.6927 - accuracy: 0.5130 - val_loss: 0.6921 - val_accuracy: 0.4906
Epoch 2/50
40/40 [==============================] - 1s 28ms/step - loss: 0.6901 - accuracy: 0.5755 - val_loss: 0.6878 - val_accuracy: 0.7500
Epoch 3/50
40/40 [==============================] - 1s 30ms/step - loss: 0.6838 - accuracy: 0.7013 - val_loss: 0.6791 - val_accuracy: 0.7386
Epoch 4/50
40/40 [==============================] - 1s 29ms/step - loss: 0.6694 - accuracy: 0.7581 - val_loss: 0.6584 - val_accuracy: 0.7686
Epoch 5/50
40/40 [==============================] - 1s 28ms/step - loss: 0.6416 - accuracy: 0.7796 - val_loss: 0.6273 - val_accuracy: 0.7698
Epoch 6/50
40/40 [==============================] - 1s 32ms/step - loss: 0.6027 - accuracy: 0.7941 - val_loss: 0.5844 - val_accuracy: 0.7988
Epoch 7/50
40/40 [==============================] - 1s 29ms/step - loss: 0.5574 - accuracy: 0.8118 - val_loss: 0.5406 - val_accuracy: 0.8252
Epoch 8/50
40

Now we will be evaluating the loss and accuracy of our model on testing data.

In [20]:
loss,accuracy=model.evaluate(test_data,test_labels)

157/157 [==============================] - 0s 2ms/step - loss: 0.2940 - accuracy: 0.8936


As we can see our model is giving an accuracy of 89.56% on the testing data.

# using SVM to train the data set

In [21]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Split the dataset into training and testing sets
train_data = df[:5000]
test_data = df[5000:7000]

# Convert text data into numerical feature vectors using TF-IDF vectorizer
vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(train_data['Reviews'])
y_train = train_data['Sentiment']
X_test = vectorizer.transform(test_data['Reviews'])
y_test = test_data['Sentiment']

# Train an SVM model
svm = SVC(kernel='linear', C=1.0, random_state=42)
svm.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = svm.predict(X_test)

# Evaluate the performance of the model
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification report:\n", classification_report(y_test, y_pred))
print("Accuracy score:", accuracy_score(y_test, y_pred))



Confusion matrix:
 [[861 121]
 [140 878]]
Classification report:
               precision    recall  f1-score   support

    negative       0.86      0.88      0.87       982
    positive       0.88      0.86      0.87      1018

    accuracy                           0.87      2000
   macro avg       0.87      0.87      0.87      2000
weighted avg       0.87      0.87      0.87      2000

Accuracy score: 0.8695


# using logistic regression

In [22]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


# Convert text data into numerical feature vectors using TF-IDF vectorizer
vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(train_data['Reviews'])
y_train = train_data['Sentiment']
X_test = vectorizer.transform(test_data['Reviews'])
y_test = test_data['Sentiment']

# Train a logistic regression model
lr = LogisticRegression(random_state=42)
lr.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = lr.predict(X_test)

print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification report:\n", classification_report(y_test, y_pred))
print("Accuracy score:", accuracy_score(y_test, y_pred))


Confusion matrix:
 [[855 127]
 [149 869]]
Classification report:
               precision    recall  f1-score   support

    negative       0.85      0.87      0.86       982
    positive       0.87      0.85      0.86      1018

    accuracy                           0.86      2000
   macro avg       0.86      0.86      0.86      2000
weighted avg       0.86      0.86      0.86      2000

Accuracy score: 0.862


In [23]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


# Tokenize the text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data['Reviews'])
X_train = tokenizer.texts_to_sequences(train_data['Reviews'])
X_test = tokenizer.texts_to_sequences(test_data['Reviews'])

# Pad the sequences to make them of equal length
maxlen = 100
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

# Convert sentiment labels to binary
y_train = np.asarray(train_data['Sentiment'].astype('category').cat.codes)
y_test = np.asarray(test_data['Sentiment'].astype('category').cat.codes)

# Build the LSTM model
embedding_size = 32
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=embedding_size, input_length=maxlen))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
batch_size = 32
epochs = 10
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

# Evaluate the model on the testing set
y_pred = (model.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))
print("Accuracy score:", accuracy_score(y_test, y_pred))

Epoch 1/10
125/125 [==============================] - 8s 50ms/step - loss: 0.6828 - accuracy: 0.5765 - val_loss: 0.6585 - val_accuracy: 0.6520
Epoch 2/10
125/125 [==============================] - 6s 49ms/step - loss: 0.4898 - accuracy: 0.7770 - val_loss: 0.4367 - val_accuracy: 0.8150
Epoch 3/10
125/125 [==============================] - 6s 48ms/step - loss: 0.2752 - accuracy: 0.8938 - val_loss: 0.4440 - val_accuracy: 0.8280
Epoch 4/10
125/125 [==============================] - 6s 49ms/step - loss: 0.1854 - accuracy: 0.9305 - val_loss: 0.4726 - val_accuracy: 0.8250
Epoch 5/10
125/125 [==============================] - 7s 59ms/step - loss: 0.1294 - accuracy: 0.9572 - val_loss: 0.4447 - val_accuracy: 0.8200
Epoch 6/10
125/125 [==============================] - 9s 73ms/step - loss: 0.0860 - accuracy: 0.9750 - val_loss: 0.5295 - val_accuracy: 0.8270
Epoch 7/10
125/125 [==============================] - 11s 89ms/step - loss: 0.0574 - accuracy: 0.9850 - val_loss: 0.5596 - val_accuracy: 0.817

In [24]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

# Load the dataset
data = reviews

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['Reviews'], data['Sentiment'], test_size=0.3, random_state=42)

# Vectorize the text data
vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2))
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Define the base classifiers
nb = MultinomialNB()
lr = LogisticRegression()
svc = LinearSVC()

# Define the ensemble classifier using voting
ensemble = VotingClassifier(estimators=[('nb', nb), ('lr', lr), ('svc', svc)], voting='hard')

# Train the ensemble classifier on the training set
ensemble.fit(X_train, y_train)

y_pred = ensemble.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Accuracy: {accuracy}")
print(f"F1 score: {f1}")


Accuracy: 0.8818333333333334
F1 score: 0.8818291479614957
